In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed()


In [10]:
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
print(housing.data.shape)
m, n = housing.data.shape
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
print(scaled_housing_data_plus_bias.shape)
print(housing.target.shape)

(20640, 8)
(20640, 9)
(20640,)


In [4]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch


In [5]:
reset_graph()

from datetime import datetime
now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
log_dir_root = 'tb_test'
logdir = '{}/run-{}'.format(log_dir_root, now)

In [11]:
reset_graph()


n_epochs = 10
lr = 1e-2
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name = 'X')
y = tf.placeholder(tf.float32, shape=(None, 1), name = 'y')
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0, seed=42), name = 'theta')
y_pred = tf.matmul(X, theta, name = 'pred')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lr)
train_op = optimizer.minimize(mse)
init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0 :
                summary_str = mse_summary.eval(feed_dict={X : X_batch, y : y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(train_op, feed_dict = {X : X_batch, y : y_batch})
    


# same code but tensorboard not working ~0327 nn #